# Task 1

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    StoppingCriteriaList,
    MaxLengthCriteria,
)
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

model.generation_config.pad_token_id = model.generation_config.eos_token_id

maxLength = 30

prompt = "Today I believe we can finally"

# return Pytorch tensor
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids

In [2]:
# Greedy Search
greedy_output = model.generate(
    input_ids, 
    max_new_tokens=maxLength, 
    return_dict_in_generate=True, 
    output_scores=True
)

# Beam Search
beam_output = model.generate(
    input_ids, 
    max_new_tokens=maxLength, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True,
    return_dict_in_generate=True, 
    output_scores=True
)

# Top-K
# set top_k to 50
top_k_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_new_tokens=maxLength, 
    top_k=50,
    return_dict_in_generate=True, 
    output_scores=True
)

# Top-P (nuclear sampling)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
top_p_output = model.generate(
    input_ids,
    do_sample=True, 
    max_new_tokens=maxLength, 
    top_k=50, 
    top_p=0.95, 
    return_dict_in_generate=True, 
    output_scores=True
)



D:\Anaconda3\lib\site-packages\transformers\generation\utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behav

### Print outputs

In [3]:
print(f'Greedy Search',tokenizer.batch_decode(greedy_output[0], skip_special_tokens=True))
print(f'\nBeam Search',tokenizer.batch_decode(beam_output[0], skip_special_tokens=True))
print(f'\nTop-K',tokenizer.batch_decode(top_k_output[0], skip_special_tokens=True))
print(f'\nTop-P',tokenizer.batch_decode(top_p_output[0], skip_special_tokens=True))

Greedy Search ['Today I believe we can finally get to the point where we can make a difference in the lives of the people of the United States of America.\n\nI believe that we can']

Beam Search ['Today I believe we can finally get to the bottom of this issue.\n\n"We need to find a way to make sure that we don\'t get into a situation where we']

Top-K ['Today I believe we can finally go forward in a political order based on peace and democracy. The people have fought too hard for this country to have a dictatorship that makes us pay taxes']

Top-P ["Today I believe we can finally bring it forward. If anything it's been far more difficult.\n\n\nIt's been too long. My experience with this has been very, very"]


### Generate Scores

In [4]:
input_length = inputs.input_ids.shape[1]
# Greedy
greedy_output_scores = model.compute_transition_scores(
    greedy_output.sequences, greedy_output.scores, normalize_logits=True
)
greedy_output_tokens = greedy_output.sequences[:, input_length:]

# Beam
beam_output_scores = model.compute_transition_scores(
    beam_output.sequences, beam_output.scores, normalize_logits=True
)
beam_output_tokens = beam_output.sequences[:, input_length:]

# Top-K
top_k_output_scores = model.compute_transition_scores(
    top_k_output.sequences, top_k_output.scores, normalize_logits=True
)
top_k_output_tokens = top_k_output.sequences[:, input_length:]

# Top-P
top_p_output_scores = model.compute_transition_scores(
    top_p_output.sequences, top_p_output.scores, normalize_logits=True
)
top_p_output_tokens = top_p_output.sequences[:, input_length:]

# for tok, score in zip(greedy_output_tokens[0], greedy_output_scores[0]):
#     # | token | token string | logits | probability
#     print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")


D:\Anaconda3\lib\site-packages\transformers\generation\utils.py:1029: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:634.)
  beam_indices[beam_indices_mask] = 0
D:\Anaconda3\lib\site-packages\transformers\generation\utils.py:1029: UserWarning: Use of masked_fill_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[mask] = scalar (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorAdvancedIndexing.cpp:1654.)
  beam_indices[beam_indices_mask] = 0


In [5]:
# Define function for perplexity and likelihood
def calculate_perplexity_and_likelihood(scores):
    # compute the sum of the log-likelihoods
    perplexity = 0
    likelihood = 0
    for score in scores[0]:
        perplexity += score.numpy()
        likelihood += np.exp(perplexity)
    perplexity = perplexity / len(scores[0])    
    return perplexity, likelihood

In [6]:
# calculate the perplexity and likelihood for the greedy output
greedy_perplexity, greedy_likelihood = calculate_perplexity_and_likelihood(greedy_output_scores)
print(f"Greedy perplexity: {greedy_perplexity:.3f}")
print(f"Greedy likelihood: {greedy_likelihood:.2%}")

# calculate the perplexity and likelihood for the beam output
beam_perplexity, beam_likelihood = calculate_perplexity_and_likelihood(beam_output_scores)
print(f"Beam perplexity: {beam_perplexity:.3f}")
print(f"Beam likelihood: {beam_likelihood:.2%}")

# calculate the perplexity and likelihood for the top-k output
top_k_perplexity, top_k_likelihood = calculate_perplexity_and_likelihood(top_k_output_scores)
print(f"Top-k perplexity: {top_k_perplexity:.3f}")
print(f"Top-k likelihood: {top_k_likelihood:.2%}")

# calculate the perplexity and likelihood for the top-p output
top_p_perplexity, top_p_likelihood = calculate_perplexity_and_likelihood(top_p_output_scores)
print(f"Top-p perplexity: {top_p_perplexity:.3f}")
print(f"Top-p likelihood: {top_p_likelihood:.2%}")

Greedy perplexity: -1.346
Greedy likelihood: 7.93%
Beam perplexity: -6.135
Beam likelihood: 7.48%
Top-k perplexity: -2.635
Top-k likelihood: 1.72%
Top-p perplexity: -2.514
Top-p likelihood: 4.65%


# Task 2

### Samsum: https://huggingface.co/datasets/samsum


In [12]:
# To load_dataset("samsum")
# !pip install py7zr

In [9]:
from datasets import load_dataset

dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to C:/Users/Jeonghoon Kim/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")

model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")

In [ ]:
import pandas as pd

# create an empty DataFrame with 5 columns
df = pd.DataFrame(columns=['prompt', 'greedy_search', 'beam_search', 'top_k', 'top_p'])


# return Pytorch tensor
maxLength = 150

# Create summary from first 50 with test set 
for i in range(50):
    prompt = dataset['test'][i]['dialogue']
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids

    # Greedy Search
    greedy_output = model.generate(
        input_ids, 
        max_new_tokens=maxLength, 
        return_dict_in_generate=True, 
        output_scores=True
    )

    # Beam Search
    beam_output = model.generate(
        input_ids, 
        max_new_tokens=maxLength, 
        num_beams=5, 
        no_repeat_ngram_size=2, 
        early_stopping=True,
        return_dict_in_generate=True, 
        output_scores=True
    )

    # Top-K
    # set top_k to 50
    top_k_output = model.generate(
        input_ids, 
        do_sample=True, 
        max_new_tokens=maxLength, 
        top_k=50,
        return_dict_in_generate=True, 
        output_scores=True
    )

    # Top-P (nuclear sampling)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
    top_p_output = model.generate(
        input_ids,
        do_sample=True, 
        max_new_tokens=maxLength, 
        top_k=50, 
        top_p=0.95, 
        return_dict_in_generate=True, 
        output_scores=True
    )
    greedy = tokenizer.batch_decode(greedy_output[0], skip_special_tokens=True)
    beam = tokenizer.batch_decode(beam_output[0], skip_special_tokens=True)
    top_k = tokenizer.batch_decode(top_k_output[0], skip_special_tokens=True)
    top_p = tokenizer.batch_decode(top_p_output[0], skip_special_tokens=True)
    
    row = {'prompt': prompt, 'greedy_search': greedy, 'beam_search': beam, 'top_k': top_k, 'top_p': top_p}
    df = df.append(row, ignore_index=True)
    
# save to csv file
df.to_csv('output.csv', encoding='utf-8-sig', index=False)
print('df saved!')